In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [ ]:
# Obter o Dataset diretamente dos datasets disponíveis no Keras 
# Neste exemplo os targets são irrelevantes, sendo carregadas apenas as imagens 28*28

(x_train, _), (x_test, _) = fashion_mnist.load_data()

In [ ]:
# Normalizar as imagens e preparar para o input no autoencoder

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

print(x_train.shape)
print(x_test.shape)

In [ ]:
# Criar uma versão das imagens com ruído aleatório. O fator de ruído define a força da perturbação adicionada
# Utiliza o método normal do tensorflow que devolve valores aleatórios seguindo uma distribuição normal
# https://www.tensorflow.org/api_docs/python/tf/random/normal
# Garante-se que os valores permanecem dentro do intervalo [0, 1]

noise_factor = 0.2
x_train_noisy = x_train + noise_factor * tf.random.normal(shape=x_train.shape) 
x_test_noisy = x_test + noise_factor * tf.random.normal(shape=x_test.shape) 

x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.)
x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)

In [ ]:
# Visualização de algumas imagens originais e com ruído
# Pode alterar o valor da variável start para visualizar outros exemplos

start= 10

n = 10
plt.figure(figsize=(20, 6))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.title("original")
    plt.imshow(tf.squeeze(x_test[i+start]))
    plt.gray()
    
    ax = plt.subplot(2, n, i + n + 1)
    plt.title("original + noise")
    plt.imshow(tf.squeeze(x_test_noisy[i+start]))
    plt.gray()
    
plt.show()

In [ ]:
# Completar a secção do Decoder. Deve utilizar camadas Conv2DTranspose parta efetuar o upsampling
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose

class Denoise(Model):
  def __init__(self):
    super(Denoise, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(28, 28, 1)),
      layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

    self.decoder = tf.keras.Sequential([
      
      # Completar com as camadas que permitam efetuar o upsampling

      layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


In [ ]:
# Criar um objeto Autoencoder

tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

autoencoder = Denoise()

In [ ]:
# Compilar o modelo
# A loss vai corresponder à diferença entre a imagem produzida e a pretendida

autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [ ]:
# Processo de treino
# O input são as imagens com ruído e o target são as imagens originais

autoencoder.fit(x_train_noisy, x_train,
                epochs=10,
                shuffle=True)

In [ ]:
# Confirmar a redução de dimensionalidade até à obtenção da representação latente
# Confirmar a expansão feita pelo Decoder até às dimensões originais

autoencoder.summary()

print('***Encoder***')
autoencoder.encoder.summary()

print('***Decoder***')
autoencoder.decoder.summary()

In [ ]:
# Avaliação do desempenho no conjunto de teste

autoencoder.evaluate(x_test_noisy, x_test)

In [ ]:
# Aplicar o modelo às imagens de teste com ruído

encoded_imgs = autoencoder.encoder(x_test_noisy).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [ ]:
# Comparar as imagens originais, as imagens com ruído e o resultado obtido pelo Autoencoder ao ser 
# alimentado com as imagens com ruído

n = 10
plt.figure(figsize=(20, 10))
for i in range(n):

    # display original
    ax = plt.subplot(3, n, i + 1)
    plt.title("Originais")
    plt.imshow(tf.squeeze(x_test[i+start]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display original + noise
    ax = plt.subplot(3, n, i + n + 1)
    plt.title("Com Ruído")
    plt.imshow(tf.squeeze(x_test_noisy[i+start]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    bx = plt.subplot(3, n, i + n*2 + 1)
    plt.title("Denoised")
    plt.imshow(tf.squeeze(decoded_imgs[i+start]))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
plt.show()